In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse



E: Package 'python-software-properties' has no installation candidate


In [0]:
from google.colab import auth 
auth.authenticate_user()

In [0]:
from oauth2client.client import GoogleCredentials 
creds = GoogleCredentials.get_application_default()

import getpass

!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL

vcode = getpass.getpass()

!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

··········


In [0]:
!pip install keras

In [0]:
!pip show tensorflow

Name: tensorflow
Version: 1.13.1
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: opensource@google.com
License: Apache 2.0
Location: /usr/local/lib/python3.6/dist-packages
Requires: tensorflow-estimator, astor, numpy, tensorboard, grpcio, protobuf, wheel, keras-preprocessing, termcolor, absl-py, gast, six, keras-applications
Required-by: stable-baselines, magenta, fancyimpute


In [0]:
!mkdir -p drive

!google-drive-ocamlfuse drive

fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option


In [0]:
print ('Files in Drive:')

!ls drive/


Files in Drive:
 558		     'Hidden Markov models for malware classification.odt'
 A4-Almost.zip	      Paper.pdf
 A4-new.zip	     'Spring 19'
 A4.zip		      Temp
 Archive	     'Tennessee (copy).zip'
 CDA.odt	     'Untitled document.odt'
'Colab Notebooks'    'Untitled spreadsheet.ods'
'CS 586.odt'	     'Web developer - ONCAMPUS.odt'
'CS586 Project.odt'


Handling Tweets

Preprocessing
|

In [0]:
import pandas as pd

gen_tweet = pd.read_csv('drive/558/gen.csv')

gen_tweet.columns

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Index(['id', 'text', 'source', 'user_id', 'truncated', 'in_reply_to_status_id',
       'in_reply_to_user_id', 'in_reply_to_screen_name', 'retweeted_status_id',
       'geo', 'place', 'contributors', 'retweet_count', 'reply_count',
       'favorite_count', 'favorited', 'retweeted', 'possibly_sensitive',
       'num_hashtags', 'num_urls', 'num_mentions', 'created_at', 'timestamp',
       'crawled_at', 'updated'],
      dtype='object')

In [0]:
len(gen_tweet.columns)

8

In [0]:
gen_tweet.columns

gen_tweet = gen_tweet[['retweet_count', 'reply_count', 'favorite_count', 'num_hashtags', 'num_urls', 'num_mentions', 
                     'text']]

gen_tweet['bot'] = 0 

f1 = pd.read_csv('drive/558/f1.csv')
f2 = pd.read_csv('drive/558/f2.csv')
f3 = pd.read_csv('drive/558/f3.csv')


f1 = f1[['retweet_count', 'reply_count', 'favorite_count', 'num_hashtags', 'num_urls', 'num_mentions', 
                     'text']]

f2 = f2[['retweet_count', 'reply_count', 'favorite_count', 'num_hashtags', 'num_urls', 'num_mentions', 
                     'text']]

f3 = f3[['retweet_count', 'reply_count', 'favorite_count', 'num_hashtags', 'num_urls', 'num_mentions', 
                     'text']]


f1['bot'] = 1 
f2['bot'] = 1 
f3['bot'] = 1 

twt = [gen_tweet, f1, f2, f3]

twt = pd.concat(twt)

twt = twt.fillna(0)



# gen_tweet.shape

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (8,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
! pip install tweet-preprocessor


  Stored in directory: /root/.cache/pip/wheels/1b/27/cc/49938e98a2470802ebdefae9d2b3f524768e970c1ebbe2dc4a
Successfully built tweet-preprocessor


In [0]:
import preprocessor as p

gen_tweet = twt

In [0]:
gen_tweet['text'] = gen_tweet['text'].apply(str)
gen_tweet['text'] = gen_tweet['text'].apply(p.clean)

In [0]:
import numpy as np

In [0]:
labels = gen_tweet['bot']

In [0]:
from keras.preprocessing.text import Tokenizer

In [0]:
t = Tokenizer()
t.fit_on_texts(gen_tweet['text'])

In [0]:
vocab_size = len(t.word_index) + 1

In [0]:
encoded_docs = t.texts_to_sequences(gen_tweet['text'])


from keras.preprocessing.sequence import pad_sequences

max_length = 4
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
# print(padded_docs)

In [0]:
embeddings_index = dict()
f = open('drive/558/glove.twitter.27B.100d.txt')

for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Loaded %s word vectors.' % len(embeddings_index))

embedding_matrix = np.zeros((vocab_size, 100))

Loaded 1193514 word vectors.


In [0]:
for word, i in t.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [0]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding

model = Sequential()
e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=4, trainable=False)
model.add(e)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# summarize the model
print(model.summary())
# fit the model
model.fit(padded_docs, labels, epochs=10, verbose=0)
# evaluate the model
loss, accuracy = model.evaluate(padded_docs, labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 4, 100)            86092500  
_________________________________________________________________
flatten_2 (Flatten)          (None, 400)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 401       
Total params: 86,092,901
Trainable params: 401
Non-trainable params: 86,092,500
_________________________________________________________________
None
Accuracy: 87.630581


In [0]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten, Reshape
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.utils import np_utils
from keras.layers.recurrent import LSTM
from keras.models import load_model

# from keras.layers import Merge

# from keras.layers import Merge

model = Sequential()
e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=4, trainable=False)
model.add(e)

model.add(LSTM(32, return_sequences=True))
model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))

model.add(Dense(1, activation='sigmoid'))

# model.add(Flatten())
# model.add(Dense(1, activation='relu'))



# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# summarize the model
print(model.summary())
# fit the model
model.fit(padded_docs, labels, epochs=1, verbose=0)
# evaluate the model
loss, accuracy = model.evaluate(padded_docs, labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 4, 100)            86092500  
_________________________________________________________________
lstm_9 (LSTM)                (None, 4, 32)             17024     
_________________________________________________________________
flatten_10 (Flatten)         (None, 128)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 128)               16512     
_________________________________________________________________
dense_11 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 65        
Total params: 86,134,357
Trainable params: 41,857
Non-trainable params: 86,092,500
___________________________________________________________